In [1]:
!pip install python-dotenv # como instalo dotenv?
!pip install langchain
!pip install langchain-openai
!pip install langchain-chroma
!pip install wikipedia
!pip install pypdf
!pip install pinecone
!pip install faiss-cpu
!pip install lark
!pip install rank_bm25
!pip install langchain_community
!pip install langchain_experimental
!pip install langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import os
from dotenv import load_dotenv, find_dotenv

In [11]:
load_dotenv("./.env") #load_dotenv(find_dotenv(), everride = True)
# Se cargan las credenciales

dotenv_path = "/content/drive/My Drive/ANALITICA_NOVIEMBRE/LANGCHAIN/.env"
load_dotenv(dotenv_path)
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [12]:
# Se cargan los LLMs

from langchain_openai  import ChatOpenAI # Para chat models
from langchain_openai import OpenAI # Para completion models

In [13]:
llm = ChatOpenAI()
output = llm.invoke("Cual es la capital de Argentina") # model = "gpt-3.5-turbo"
print(output.content)

La capital de Argentina es Buenos Aires.


# 1- INTRODUCCION

In [14]:
from langchain.memory import ConversationBufferMemory

* Ejemplos con Con `add_user_message` y `add_ai_message`.
* Se observa que se retorna una sola `clave` denominada `history`, la cual contiene dos mensajes, el del `usuario` y el del `LLM`.

In [15]:
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

memory.load_memory_variables({})

{'history': "Human: hi!\nAI: what's up?"}

Con `save_context`:

In [16]:
memory = ConversationBufferMemory()
memory.save_context({"input": "hi"}, {"output": "whats up"})

memory.load_memory_variables({})

{'history': 'Human: hi\nAI: whats up'}

Se puede cambiar el nombre a la variable de salida

In [17]:
memory = ConversationBufferMemory(memory_key = "chat_history")
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

In [18]:
memory.load_memory_variables({})

{'chat_history': "Human: hi!\nAI: what's up?"}

## A- Completion Model

Cuando se quiere trabajar con un modelo de `completion` es útil tener como salida un `string`, dado que eso es lo que espera el modelo.

In [19]:
# Como en el ejemplo anterior, se devuelve un string para el modelo de completion

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

memory.load_memory_variables({})

{'history': "Human: hi!\nAI: what's up?"}

In [20]:
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory


llm = OpenAI(temperature = 0)

template = """You are a nice chatbot having a conversation with a human.

Previous conversation:
{chat_history}

New human question: {question}
Response:"""

prompt = PromptTemplate.from_template(template)

memory = ConversationBufferMemory(memory_key = "chat_history")

conversation = LLMChain(
    llm = llm,
    prompt = prompt,
    verbose = True,
    memory = memory
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [21]:
conversation.invoke({"question": "hi"})



> Entering new LLMChain chain...
Prompt after formatting:
You are a nice chatbot having a conversation with a human.

Previous conversation:


New human question: hi
Response:

> Finished chain.


{'question': 'hi',
 'chat_history': '',
 'text': ' Hello! How are you doing today?'}

In [22]:
conversation({"question": "how are you?"})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
You are a nice chatbot having a conversation with a human.

Previous conversation:
Human: hi
AI:  Hello! How are you doing today?

New human question: how are you?
Response:

> Finished chain.


{'question': 'how are you?',
 'chat_history': 'Human: hi\nAI:  Hello! How are you doing today?',
 'text': " I am a chatbot, so I don't have feelings like humans do. But I am programmed to be helpful and friendly. Is there anything I can assist you with?"}

## B- Chat Model

Cuando se quiere trabajar con un modelo de `chat` es útil tener como salida una `lista` de `chat messages`.

In [23]:
# En este ejemplo se devuelve como una lista de mensajes

memory = ConversationBufferMemory(return_messages = True) # Esta es la unica variable que hay que modificar
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

memory.load_memory_variables({})

{'history': [HumanMessage(content='hi!'), AIMessage(content="what's up?")]}

In [24]:
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory


llm = ChatOpenAI()
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        MessagesPlaceholder(variable_name = "chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

memory = ConversationBufferMemory(memory_key = "chat_history", return_messages = True)

conversation = LLMChain(
    llm = llm,
    prompt = prompt,
    verbose = True,
    memory = memory
)

In [25]:
conversation({"question": "hi"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi

> Finished chain.


{'question': 'hi',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='Hello! How are you doing today?')],
 'text': 'Hello! How are you doing today?'}

In [26]:
conversation({"question": "I'm fine and you?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: Hello! How are you doing today?
Human: I'm fine and you?

> Finished chain.


{'question': "I'm fine and you?",
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='Hello! How are you doing today?'),
  HumanMessage(content="I'm fine and you?"),
  AIMessage(content="I'm doing well, thank you for asking. Is there anything on your mind that you'd like to talk about or ask?")],
 'text': "I'm doing well, thank you for asking. Is there anything on your mind that you'd like to talk about or ask?"}

## C- Un último ejemplo end-to-end

**Con modelo completion**

In [27]:
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

In [28]:
template = """You are a nice chatbot having a conversation with a human.

Previous conversation:
{chat_history}

New human question: {question}
Response:"""

In [29]:
prompt = PromptTemplate.from_template(template)

In [30]:
memory = ConversationBufferMemory(memory_key="chat_history")

In [31]:
llm = OpenAI(temperature=0)

In [32]:
conversation = LLMChain(
    llm = llm,
    prompt = prompt,
    verbose = True,
    memory = memory
)

In [33]:
conversation({"question": "hi"})



> Entering new LLMChain chain...
Prompt after formatting:
You are a nice chatbot having a conversation with a human.

Previous conversation:


New human question: hi
Response:

> Finished chain.


{'question': 'hi',
 'chat_history': '',
 'text': ' Hello! How are you doing today?'}

In [34]:
conversation({"question": "My name is Javier!"})



> Entering new LLMChain chain...
Prompt after formatting:
You are a nice chatbot having a conversation with a human.

Previous conversation:
Human: hi
AI:  Hello! How are you doing today?

New human question: My name is Javier!
Response:

> Finished chain.


{'question': 'My name is Javier!',
 'chat_history': 'Human: hi\nAI:  Hello! How are you doing today?',
 'text': ' Nice to meet you, Javier! My name is AI. How can I assist you today?'}

In [35]:
conversation({"question": "What's my name?"})



> Entering new LLMChain chain...
Prompt after formatting:
You are a nice chatbot having a conversation with a human.

Previous conversation:
Human: hi
AI:  Hello! How are you doing today?
Human: My name is Javier!
AI:  Nice to meet you, Javier! My name is AI. How can I assist you today?

New human question: What's my name?
Response:

> Finished chain.


{'question': "What's my name?",
 'chat_history': 'Human: hi\nAI:  Hello! How are you doing today?\nHuman: My name is Javier!\nAI:  Nice to meet you, Javier! My name is AI. How can I assist you today?',
 'text': ' Your name is Javier.'}

**Con chat model**

In [36]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

In [37]:
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

In [38]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [39]:
conversation = LLMChain(
    llm = llm,
    prompt = prompt,
    verbose = True,
    memory = memory
)

In [40]:
llm = ChatOpenAI()

In [41]:
conversation({"question": "hi"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi

> Finished chain.


{'question': 'hi',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='\nSystem: Hello! How are you doing today?')],
 'text': '\nSystem: Hello! How are you doing today?'}

In [42]:
conversation({"question": "My name is Javier!"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: 
System: Hello! How are you doing today?
Human: My name is Javier!

> Finished chain.


{'question': 'My name is Javier!',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='\nSystem: Hello! How are you doing today?'),
  HumanMessage(content='My name is Javier!'),
  AIMessage(content='\nAI: Nice to meet you, Javier! I am an AI chatbot designed to have conversations with humans. What would you like to talk about?')],
 'text': '\nAI: Nice to meet you, Javier! I am an AI chatbot designed to have conversations with humans. What would you like to talk about?'}

In [43]:
conversation({"question": "What's my name?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: 
System: Hello! How are you doing today?
Human: My name is Javier!
AI: 
AI: Nice to meet you, Javier! I am an AI chatbot designed to have conversations with humans. What would you like to talk about?
Human: What's my name?

> Finished chain.


{'question': "What's my name?",
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='\nSystem: Hello! How are you doing today?'),
  HumanMessage(content='My name is Javier!'),
  AIMessage(content='\nAI: Nice to meet you, Javier! I am an AI chatbot designed to have conversations with humans. What would you like to talk about?'),
  HumanMessage(content="What's my name?"),
  AIMessage(content='\nAI: Your name is Javier, as you mentioned earlier. Is there anything else you would like to know?')],
 'text': '\nAI: Your name is Javier, as you mentioned earlier. Is there anything else you would like to know?'}

# 2- TIPOS DE MEMORIA

## A- ConversationBufferMemory

* `ConversationBufferMemory` es un tipo de memoria que simplemente mantiene una lista de `chat messages` en un búfer.
* Cuando se hace un llamado al `LLM` la memoria se pasa como parte del `PromptTemplate`.

Es el ejemplo anterior:

In [44]:
llm = ChatOpenAI()
prompt = ChatPromptTemplate(messages =
    [
        SystemMessagePromptTemplate.from_template(template = "You are a nice chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name = "chat_history"),
        HumanMessagePromptTemplate.from_template(template = "{question}")
    ]
)

memory = ConversationBufferMemory(memory_key = "chat_history", return_messages = True)

conversation = LLMChain(
    llm = llm,
    prompt = prompt,
    verbose = True,
    memory = memory
)

In [45]:
conversation({"question": "How are you?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: How are you?

> Finished chain.


{'question': 'How are you?',
 'chat_history': [HumanMessage(content='How are you?'),
  AIMessage(content="I'm doing well, thank you for asking! How about you? How are you feeling today?")],
 'text': "I'm doing well, thank you for asking! How about you? How are you feeling today?"}

In [46]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='How are you?'),
  AIMessage(content="I'm doing well, thank you for asking! How about you? How are you feeling today?")]}

In [47]:
conversation({"question": "I'm great! How much is 2 + 2?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: How are you?
AI: I'm doing well, thank you for asking! How about you? How are you feeling today?
Human: I'm great! How much is 2 + 2?

> Finished chain.


{'question': "I'm great! How much is 2 + 2?",
 'chat_history': [HumanMessage(content='How are you?'),
  AIMessage(content="I'm doing well, thank you for asking! How about you? How are you feeling today?"),
  HumanMessage(content="I'm great! How much is 2 + 2?"),
  AIMessage(content="2 + 2 equals 4! Do you have any other questions or topics you'd like to talk about?")],
 'text': "2 + 2 equals 4! Do you have any other questions or topics you'd like to talk about?"}

In [48]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='How are you?'),
  AIMessage(content="I'm doing well, thank you for asking! How about you? How are you feeling today?"),
  HumanMessage(content="I'm great! How much is 2 + 2?"),
  AIMessage(content="2 + 2 equals 4! Do you have any other questions or topics you'd like to talk about?")]}

In [49]:
conversation({"question": "And 2 more?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: How are you?
AI: I'm doing well, thank you for asking! How about you? How are you feeling today?
Human: I'm great! How much is 2 + 2?
AI: 2 + 2 equals 4! Do you have any other questions or topics you'd like to talk about?
Human: And 2 more?

> Finished chain.


{'question': 'And 2 more?',
 'chat_history': [HumanMessage(content='How are you?'),
  AIMessage(content="I'm doing well, thank you for asking! How about you? How are you feeling today?"),
  HumanMessage(content="I'm great! How much is 2 + 2?"),
  AIMessage(content="2 + 2 equals 4! Do you have any other questions or topics you'd like to talk about?"),
  HumanMessage(content='And 2 more?'),
  AIMessage(content="Adding 2 more to 4 gives you a total of 6! Is there anything else you'd like to know or discuss?")],
 'text': "Adding 2 more to 4 gives you a total of 6! Is there anything else you'd like to know or discuss?"}

## B- ConversationBufferWindowMemory

- Sólo utiliza las últimas `k` interacciones.
- Esto puede resultar útil pare evitar que el buffer crezca demasiado.

In [50]:
from langchain.memory import ConversationBufferWindowMemory

In [51]:
llm = ChatOpenAI()
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        MessagesPlaceholder(variable_name = "chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

In [52]:
memory = ConversationBufferWindowMemory(memory_key = "chat_history", k = 3, return_messages = True)
memory.save_context({"input": "Hi how are you?"}, {"output": "I'm fine thanks."})
memory.save_context({"input": "How much is 2 + 2?"}, {"output": "It's 4."})
memory.save_context({"input": "And 2 more?"}, {"output": "With 2 more It's 6."})

memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='Hi how are you?'),
  AIMessage(content="I'm fine thanks."),
  HumanMessage(content='How much is 2 + 2?'),
  AIMessage(content="It's 4."),
  HumanMessage(content='And 2 more?'),
  AIMessage(content="With 2 more It's 6.")]}

Ejemplo

In [53]:
llm = ChatOpenAI()
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        MessagesPlaceholder(variable_name = "chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

memory = ConversationBufferWindowMemory(memory_key = "chat_history", k = 4, return_messages = True)

In [54]:
conversation = LLMChain(
    llm = llm,
    prompt = prompt,
    memory = memory
)

In [55]:
conversation({"question": "Hi my name is Javier!"})["text"]

"Hello Javier! It's nice to meet you. How are you today?"

In [56]:
conversation({"question": "What is the mass of the Earth?"})["text"]

"The mass of the Earth is approximately 5.97 x 10^24 kilograms. It's amazing to think about how massive our planet is, isn't it?"

In [57]:
conversation({"question": "And it's diameter"})["text"]

"The diameter of the Earth is about 12,742 kilometers. Earth is the largest of the four terrestrial planets in our solar system. It's fascinating to learn more about our home planet, don't you think?"

In [58]:
conversation({"question": "What is my name?"})["text"]

"Your name is Javier, as you mentioned earlier. It's always nice to have a conversation with you, Javier!"

## C- ConversationSummaryMemory

- Este tipo de memoria crea un resumen de la conversación a lo largo del tiempo.
- Resume la conversación a medida que ocurre y almacena el resumen actual en la memoria.
- Mantener el historial de mensajes pasados palabra consume demasiados tokens.

In [59]:
from langchain.memory import ConversationSummaryMemory

In [60]:
llm = ChatOpenAI()
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        MessagesPlaceholder(variable_name = "chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

memory = ConversationSummaryMemory(llm = llm, memory_key = "chat_history", return_messages = True)

conversation = LLMChain(
    llm = llm,
    prompt = prompt,
    memory = memory
)

In [61]:
print(conversation.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


In [62]:
conversation({"question": "Good morning AI!"})

{'question': 'Good morning AI!',
 'chat_history': [SystemMessage(content='')],
 'text': 'Good morning! How are you today?'}

In [63]:
conversation({"question": "I'm fine and you! Tell me a joke please!"})

{'question': "I'm fine and you! Tell me a joke please!",
 'chat_history': [SystemMessage(content='The human greets the AI with a "Good morning". The AI responds with a friendly "Good morning" and asks how the human is feeling today.')],
 'text': "I'm glad to hear you're feeling fine! Here's a joke for you: Why did the math book look sad? Because it had too many problems. I hope that brought a smile to your face!"}

In [64]:
print(conversation.memory.buffer)

The human greets the AI with a "Good morning" and the AI responds in kind, asking how the human is feeling. The human is feeling fine and asks the AI to tell a joke, to which the AI responds with a math joke and hopes it brought a smile to the human's face.


In [65]:
print(memory.load_memory_variables({})["chat_history"])

[SystemMessage(content='The human greets the AI with a "Good morning" and the AI responds in kind, asking how the human is feeling. The human is feeling fine and asks the AI to tell a joke, to which the AI responds with a math joke and hopes it brought a smile to the human\'s face.')]


## D- ConversationSummaryBufferMemory

In [66]:
from langchain.memory import ConversationSummaryBufferMemory

In [67]:
llm = ChatOpenAI()
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        MessagesPlaceholder(variable_name = "chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

memory = ConversationSummaryBufferMemory(llm = llm, memory_key = "chat_history", return_messages = True, max_token_limit = 650)

conversation = LLMChain(
    llm = llm,
    prompt = prompt,
    memory = memory
)

In [68]:
print(conversation.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


In [69]:
conversation({"question": "Good morning AI!"})

{'question': 'Good morning AI!',
 'chat_history': [HumanMessage(content='Good morning AI!'),
  AIMessage(content='Good morning! How are you doing today?')],
 'text': 'Good morning! How are you doing today?'}

In [70]:
conversation({"question": "I'm fine and you! Tell me a joke please!"})

{'question': "I'm fine and you! Tell me a joke please!",
 'chat_history': [HumanMessage(content='Good morning AI!'),
  AIMessage(content='Good morning! How are you doing today?'),
  HumanMessage(content="I'm fine and you! Tell me a joke please!"),
  AIMessage(content="Sure! Here's a joke for you: Why couldn't the bicycle find its way home? Because it lost its bearings! 😄")],
 'text': "Sure! Here's a joke for you: Why couldn't the bicycle find its way home? Because it lost its bearings! 😄"}

In [71]:
print(conversation.memory.buffer)

[HumanMessage(content='Good morning AI!'), AIMessage(content='Good morning! How are you doing today?'), HumanMessage(content="I'm fine and you! Tell me a joke please!"), AIMessage(content="Sure! Here's a joke for you: Why couldn't the bicycle find its way home? Because it lost its bearings! 😄")]


In [72]:
print(memory.load_memory_variables({})["chat_history"])

[HumanMessage(content='Good morning AI!'), AIMessage(content='Good morning! How are you doing today?'), HumanMessage(content="I'm fine and you! Tell me a joke please!"), AIMessage(content="Sure! Here's a joke for you: Why couldn't the bicycle find its way home? Because it lost its bearings! 😄")]
